<a href="https://colab.research.google.com/github/JamieBali/faceAlignment/blob/main/faceAlignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites 

In [ ]:
!pip uninstall opencv-python -y
!pip install opencv-contrib-python==3.4.2.17 --force-reinstall

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn import linear_model
import cv2
from PIL import Image
from matplotlib.pyplot import imshow

from google.colab.patches import cv2_imshow


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:

# Download the data stored in a zipped numpy array from one of these two locations
# The uncommented one is likely to be faster. If you're running all your experiments
# on a machine at home rather than using colab, then make sure you save it 
# rather than repeatedly downloading it.
!wget "http://users.sussex.ac.uk/~is321/training_images.npz" -O training_images.npz
# The test images (without points)
!wget "http://users.sussex.ac.uk/~is321/test_images.npz" -O test_images.npz
# The example images are here
!wget "http://users.sussex.ac.uk/~is321/examples.npz" -O examples.npz

# Load the data using np.load
training_data = np.load('training_images.npz', allow_pickle=True)
test_data = np.load('test_images.npz', allow_pickle=True)
example_data = np.load('examples.npz', allow_pickle = True)

In [ ]:

# Extract the images
training_images = training_data['images']
test_images = test_data['images']
example_images = example_data['images']
# and the data points
training_pts = training_data['points']

training_set = training_images[:-100]     # all but the last 100 for training
training_set_pts = training_pts[:-100]

validation_set = training_images[-100:]   # the last 100 for validation
validation_set_points = training_pts[-100:]

## have a look at whether using 100 has a significant effect? It's like 2700 other images, so it should be fine but uhhhhhhhhh

In [ ]:
visualise_pts(training_set[-1], training_set_pts[-1])

In [ ]:
cumulative = []
for x in training_set_pts:
  cumulative.append(np.average(calculate_error(avg_pts, x)))

print(np.average(cumulative))

10.84346871689841


# Reusable Functions

In [ ]:
def visualise_pts(img, pts, back=None):
  plt.imshow(img, back)
  for x in pts:
    plt.plot(x[0],x[1], '+r')
  plt.show()

def calculate_error(pred_pts, gt_pts):
  """
  Calculate the euclidean distance between pairs of points
  :param pred_pts: The predicted points
  :param gt_pts: The ground truth points
  :return: An array of shape (no_points,) containing the distance of each predicted point from the ground truth
  """
  pred_pts = np.reshape(pred_pts, (-1, 2))
  gt_pts = np.reshape(gt_pts, (-1, 2))
  return np.sqrt(np.sum(np.square(pred_pts - gt_pts), axis=-1))  

def save_as_csv(points, location = '.'):
  """
  Save the points out as a .csv file
  :param points: numpy array of shape (no_image, no_points, 2) to be saved
  :param location: Directory to save results.csv in. Default to current working directory
  """
  np.savetxt(location + '/results.csv', np.reshape(points, (points.shape[0], -1)), delimiter=',')


visualise_pts(training_images[0], training_pts[0])



# Pre-Processing

In [ ]:
# First, all images need to be greyscale
grey_images = []
for x in training_set:
  grey_images.append(np.uint8(np.mean(x, axis=-1)))     # reduce all images to single axis by averaging pixel intensities

# Then we need to get the average image
avg_image = np.average(grey_images, axis = 0)       # and average all those images to get the mean face

# And then the average markers
avg_pts = []
for x in range(46):                          # for each of the markers
  px = np.average(training_set_pts[:,x,0])    # average every x coord for that marker
  py = np.average(training_set_pts[:,x,1])    # average every y coord for that marker
  temp = [px, py]
  avg_pts.append(temp) 

def pre_processing(image):
  # image = np.uint8(np.mean(image, axis=-1))        # greyscale       # removed because it made it run worse?
  # norm_img = np.zeros((236,236))
  # norm_img = cv2.normalize(image, norm_img, 0, 255, cv2.NORM_MINMAX) # seems to have no effect and just increases run time
  blur_img = cv2.GaussianBlur(image,(5,5),0)
  resized_image = resize_image(blur_img)
  return resized_image

def resize_image(image):
  return(cv2.resize(image, (118,118), interpolation= cv2.INTER_AREA))   # reducing size to 118 (scaling factor of 0.5)

def resize_pts(pts):
  resized_pts = []
  for x in pts:                          # for each point, half it's coords
    rx = x[0] / 2
    ry = x[1] / 2
    resized_pts.append([rx, ry]) # append in the correct format
  return resized_pts

def upscale(pts):
  resized_pts = []           
  for x in pts:         # for each point, double it's coords
    rx = x[0] * 2
    ry = x[1] * 2
    resized_pts.append([rx, ry])
  return resized_pts

visualise_pts(avg_image, avg_pts, 'gray')
cv2_imshow(pre_processing(training_set[1]))


In [ ]:
# To get more training data, we're gonna flip all the images. ### NO LONGER IN USE ###
#flipped_images = []
#flipped_pts = []

#for x in range(len(training_set)):
#  flipped_images.append(np.fliplr(np.copy(training_set[x])))

#flipped_images = np.array(flipped_images)  # has to be a numpy array becuase apparently lists just aren't good enough -_-

#for x in range(len(training_set_pts)):
#  retPts = []
#  ys = []                                   
#  xs = []
#  for p in training_set_pts[x]:
#    ys.append(p[1])                 # y coords don't need to change
#    xs.append(236 - p[0])           # invert the x coords
#  for y in range(len(xs)):
#    retPts.append([xs[y],ys[y]])    # and correctly reformat
#  flipped_pts.append(retPts)

#actually_flipped_pts = []
#for x in flipped_pts:
#   actually_flipped_pts.append(reorderList(x))    # and reorder them.

#actually_flipped_pts = np.array(actually_flipped_pts)        # and turn this one into a numpy array too

#training_set_pts = np.concatenate((training_set_pts, actually_flipped_pts))        ##
#training_set = np.concatenate((training_set, flipped_images))                      # add both of these too the training set

#visualise_pts(flipped_images[1], flipped_pts[1])
#visualise_pts(training_set[-1], training_set_pts[-1])

In [ ]:
#def reorderList(givenList):
#  reorder = [6,5,4,3,2,1,0,12,11,10,9,8,7,13,14,17,16,15,27,26,25,24,29,28,21,20,19,18,23,22,36,35,34,33,32,31,30,41,40,39,38,37,44,43,42,45]     # this is the mapping order
                                                                                                                                                   # so that these flipped points are
                                                                                                                                                   # correctly ordered and labeled
#  reorderedList = []
#  for i in reorder:
#    reorderedList.append(givenList[i])
#  return reorderedList

# Cascaded Regression


In [ ]:
def cascade_train(no_of_regressors, damping_factor):
  sift = cv2.xfeatures2d.SIFT_create()
  points = []
  processed_images = []
  resized_ground_points = []  

  for x in training_set_pts:                        
    temp = np.copy(avg_pts)                           # copying just in case
    points.append(resize_pts(temp))                   ##
    resized_ground_points.append(resize_pts(x))       # and then we resize the starting points and ground points

  for x in training_set:
    processed_images.append(pre_processing(x))        # preprocessing

  points = np.array(points)
  resized_ground_points = np.array(resized_ground_points)       # apparently they need to be in arrays.

  regressors = []       
  for i in range(no_of_regressors):  # for each regressor
    A  = []
    target = []
    for j in range(len(processed_images)): # for each image
      if i>=1: 
        points[j] = regressor_predict(processed_images[j],regressors[i-1], points[j], sift, damping_factor) # we don't want this to run the first itteration as there isn't a previous regressor 
                                                                                                            # to run yet. Passing the sift feature here makes it a lot faster.
    
      A.append(sift.compute(processed_images[j],get_keypoints_from_points(points[j]))[1])       # we only want the descriptors - we calculated the keypoints outselves already.
      
      target.append(resized_ground_points[j]-points[j])

    A = np.array([a.flatten() for a in A])                            # the model wants it flattened
    target = np.array([t.flatten() for t in target])                  
    model = linear_model.LinearRegression()                           
    regressors.append(model.fit(A,target))                            # fits data into a linear regressor, courtesy of sklearn
    
  return regressors

def regressor_predict(image,regressor, previous_points, sift, damping_factor = 0.15):
  points = previous_points
  a = sift.compute(image,get_keypoints_from_points(points))[1]          # once again, we only want the descriptors
  prediction = regressor.predict(a.reshape(-1,5888))                    # because we flattened the array, its size is now 5888 (46 features x 128(depth of sift features))
  prediction = prediction * damping_factor                              # using a damping factor so that we don't overshoot and start oscilating around the ground points
  returnVal = points + (prediction.reshape(-1,2))                       # we want to return points - not deltas
  return returnVal                

def get_keypoints_from_points(face_points,keypoint_size=1):
  keypoints = []
  for p in range(46): 
    x_coord = face_points[p][0]
    y_coord = face_points[p][1]
    keypoints.append(cv2.KeyPoint(x_coord,y_coord,keypoint_size))
  return keypoints

In [ ]:
regressors = cascade_train(25, 0.15)

In [ ]:
def run_regressors(image, regressors, damping_factor):
  sift = cv2.xfeatures2d.SIFT_create()
  points = resize_pts(avg_pts)
  image = pre_processing(image)
  for k in range(len(regressors)):                                                  # going through each regressor
    a = sift.compute(image,get_keypoints_from_points(points))[1]                            # calculate sift features
    prediction = regressors[k].predict(a.reshape(-1,5888))                          # and make the prediction
    prediction = prediction * damping_factor
    points = points + (prediction.reshape(-1,2))
  return upscale(points)                                                            # we need to remember to upscale the
                                                                                    # points at the end so they fit on the
                                                                                    # image correctly


In [ ]:
for x in range(len(validation_set)):
  damping_factor = 0.15
  new_points = run_regressors(validation_set[x], regressors, damping_factor)
  t = np.average(calculate_error(new_points, validation_set_points[x])) 
  if t < 4:
    visualise_pts(validation_set[x], new_points)
    print(t)

# Experiment

In [ ]:
### AN EXPERIMENT

dampingFactors = [0.001, 0.005, 0.01, 0.05, 0.075, 0.1]
numbersOfRegressors = [5, 10, 15, 20, 25]

results = {}
for factor in dampingFactors:
  for regressorCount in numbersOfRegressors:
    temp = str(factor) + "|" + str(regressorCount)
    results[temp] = []

count = 0

for factor in dampingFactors:
  for regressorCount in numbersOfRegressors:
    count += 1
    print(str(count) + " / 30" )
    regressors = cascade_train(regressorCount, factor)
    temp = str(factor) + "|" + str(regressorCount)
    for im in range(len(validation_set)):
      new_points = cascade(validation_set[im], regressors, factor)
      results[temp].append(calculateError(new_points, validation_set_points[im]))

print(results)

In [ ]:
### Second Experiment
dampingFactors = [0.1, 0.15, 0.2]
numbersOfRegressors = [25, 30, 40, 50]

results = {}
for factor in dampingFactors:
  for regressorCount in numbersOfRegressors:
    temp = str(factor) + "|" + str(regressorCount)
    results[temp] = []

count = 0

for factor in dampingFactors:
  for regressorCount in numbersOfRegressors:
    count += 1
    print(str(count) + " / 16" )
    regressors = cascade_train(regressorCount, factor)
    temp = str(factor) + "|" + str(regressorCount)
    for im in range(len(validation_set)):
      new_points = cascade(validation_set[im], regressors, factor)
      results[temp].append(calculateError(new_points, validation_set_points[im]))

print(results)

In [ ]:
averages = {}

for x in results:
  averages[x] = np.average(results[x])

print(averages)
print(sorted(averages))

## 0.15 | 25 seems to be the best



In [ ]:
import pandas as pd

_data = {'0.1|25': 168.25347255294443, '0.1|30': 167.4106967820352, '0.1|40': 167.6372564067228, '0.1|50': 167.8401195469865, '0.15|25': 166.91659154202898, '0.15|30': 167.23857593344994, '0.15|40': 167.4855639985779, '0.15|50': 167.5440399544135, '0.2|25': 168.1771384041849, '0.2|30': 168.290513875452, '0.2|40': 168.3411044950288, '0.2|50': 168.34645097543452}
_pd = []
_indexi = []

for x in _data:
  _pd.append(np.average(_data[x]))
  _indexi.append(x)

df = pd.DataFrame(_pd, index=_indexi)
df.plot(legend=False)

In [ ]:
output_data = []
for x in test_images:
  temp_points = run_regressors(x, regressors, 0.15)
  output_data.append(temp_points)
  visualise_pts(x, temp_points)
data_to_be_saved = np.array(output_data)

# Flower Crown

In [ ]:
def applyFlowerCrown(img, pts):
  original_image = Image.fromarray(np.uint8(np.array(np.copy(img))))
  temple_to_temple = int(pts[6][0] - pts[0][0]) + 60      # this is the difference between points 0 and 6, with an adition of 60 pixels for error cases
  height = int(temple_to_temple / 2)                      # the height is half of this to count for scaling

  
  xDelta = pts[11][0] - pts[8][0]                         # this is some basic trigonometry to figure out the angle the crown needs
  yDelta = pts[11][1] - pts[8][1]                         # to rest at to look correct.
  opp_over_adj = yDelta / xDelta
  angle_between_eyebrows = np.arctan(opp_over_adj) * -57.3       # radians bad


  source_x = int(pts[8][0] - (temple_to_temple/3))             # these values have been adjusted and this seems to look best on the test cases i tried
  source_y = int(pts[8][1] - (height/1.3))

  crown = Image.open('/content/gdrive/MyDrive/flowerCrown.png')  # reimport so reruns are all reset images

  crown = crown.resize((temple_to_temple,height))
  crown = crown.rotate(angle_between_eyebrows)

  original_image.paste(crown,[source_x, source_y], mask=crown)

  return original_image

#num = 9
#applyFlowerCrown(training_set[num], training_set_pts[num])

In [ ]:
for ex_im in example_images:
  new_points = run_regressors(ex_im, regressors, damping_factor)
  display(applyFlowerCrown(ex_im, new_points))

# Save Data to Drive

In [ ]:
save_as_csv(data_to_be_saved, "/content/gdrive/MyDrive/")